# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
%pip install xlsxwriter

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data = requests.get(api_url).json()
data

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [ ]:
data[0]['year1ChangePercent']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [95]:
# df = pd.DataFrame(columns = my_columns)
df = pd.DataFrame()
new_df1 = pd.DataFrame()
new_df2 = pd.DataFrame()

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    half = len(data)//2
    data1, data2 = data[:half], data[half:]

    for data_arr in data1:
        new_df = pd.DataFrame(
            [[
                data_arr['symbol'],
                data_arr['latestPrice']
            ]],columns=['Ticker', 'Price']
        )
        new_df1 = pd.concat([new_df1, new_df], ignore_index=True)

    for data_arr in data2:
        new_df = pd.DataFrame(
            [[
                data_arr['year1ChangePercent']
            ]],columns=['One-Year Price Return']
        )
        new_df2 = pd.concat([new_df2, new_df], ignore_index=True)

df = new_df1.join(new_df2)
df

# 10.2s

/tmp/ipykernel_1454/3191536411.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df2 = pd.concat([new_df2, new_df], ignore_index=True)


,Ticker,Price,One-Year Price Return
0,A,132.03,-0.064054
1,AAL,15.13,-0.020566
2,AAP,61.10,-0.566740
3,AAPL,182.52,0.249653
4,ABBV,178.09,0.220542
...,...,...,...
500,YUM,138.32,0.082436
501,ZBH,129.18,0.047868
502,ZBRA,274.99,-0.083281
503,ZION,39.03,-0.153561


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [100]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df
df.reset_index(inplace=True)
df

,index,Ticker,Price,One-Year Price Return
0,345,NVDA,788.17,2.785913
1,275,LB,79.92,2.281287
2,31,AMD,176.52,1.373842
3,283,LLY,769.54,1.358412
4,50,AVGO,1296.37,1.339373
5,288,LRCX,928.50,1.020012
6,370,PHM,105.74,0.988488
7,37,ANET,267.66,0.987723
8,266,KLAC,668.82,0.844350
9,29,AMAT,197.16,0.835561


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [101]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")

portfolio_input()


10000000


In [116]:
new_df3 = pd.DataFrame()

position_size = float(portfolio_size) / len(df.index)
# # print(position_size)

for i in range(0, len(df.index)):
    # df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[i, 'Price'])
      
    new_df = pd.DataFrame(
            [[
                math.floor(position_size/df.loc[i, 'Price'])
            ]],columns=['Number of Shares to Buy']
        )
    new_df3 = pd.concat([new_df3, new_df], ignore_index=True)

df = df.join(new_df3)
df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,345,NVDA,788.17,2.785913,253
1,275,LB,79.92,2.281287,2502
2,31,AMD,176.52,1.373842,1133
3,283,LLY,769.54,1.358412,259
4,50,AVGO,1296.37,1.339373,154
5,288,LRCX,928.50,1.020012,215
6,370,PHM,105.74,0.988488,1891
7,37,ANET,267.66,0.987723,747
8,266,KLAC,668.82,0.844350,299
9,29,AMAT,197.16,0.835561,1014


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: